# 如何部分格式化 Prompt 模板

:::info 先决条件

本指南假定您已熟悉以下概念：
- [Prompt 模板](/docs/concepts/prompt_templates)

:::

如同部分绑定函数参数一样，对 [Prompt 模板](/docs/concepts/prompt_templates/) 进行“部分”操作也是有意义的——例如，传入所需值的一个子集，以此创建一个新的 Prompt 模板，该模板仅期望剩余的剩余值。

LangChain 支持两种方式来实现这一点：

1. 使用字符串值进行部分格式化。
2. 使用返回字符串值的函数进行部分格式化。

在下面的示例中，我们将探讨这两种用例的动机以及如何在 LangChain 中实现它。

## 使用字符串进行部分格式化

想要对 Prompt 模板进行部分格式化的一个常见用例是，如果您比其他变量更早地获得了 Prompt 中某些变量的访问权限。例如，假设您有一个 Prompt 模板，它需要两个变量 `foo` 和 `baz`。如果您在链的早期获得了 `foo` 的值，但在后期才获得 `baz` 的值，那么将这两个变量一直传递到链的末尾可能会很不方便。相反，您可以使用 `foo` 值对 Prompt 模板进行部分格式化，然后传递部分格式化后的 Prompt 模板，并仅使用它。下面是执行此操作的示例：

In [1]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


你也可以直接用部分变量来初始化提示词。

In [2]:
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


## 使用函数进行部分应用

另一种常见的用法是使用函数进行部分应用。这种用例适用于你有一个变量，你知道你总是希望以一种通用的方式获取它。一个典型的例子是日期或时间。想象一下你有一个提示，你总是希望它包含当前日期。你无法将其硬编码到提示中，并且与其他输入变量一起传递它很不方便。在这种情况下，能够使用一个始终返回当前日期的函数来部分应用（partial）该提示会非常方便。

In [4]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/21/2024, 19:43:57


你也可以直接用部分变量初始化提示，这在这种工作流程中通常更合理。

In [5]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/21/2024, 19:43:57


## 后续步骤

您现在已经学会了如何部分应用变量到您的提示模板。

接下来，请查看本节中关于提示模板的其他操作指南，例如 [为您的提示模板添加少量示例](/docs/how_to/few_shot_examples_chat)。